### It turns out that Epoch = 3, Batch Size = 8 , Learning Rate = 5e-5 performs well (without overfitting)

In [ ]:
!pip install datasets transformers

from google.colab import drive
drive.mount('/content/drive')

In [1]:

import pandas as pd

from datasets import Dataset
from transformers import AutoTokenizer
import tensorflow as tf
from transformers import TFAutoModelForSequenceClassification
from transformers import DefaultDataCollator

/home/lee/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
BERT_MODEL = "bert-base-uncased"
#BERT_MODEL = "vinai/bertweet-base"
NUM_EPOCHS = 6
BATCH_SIZE = 20
DROPOUT = 0.2

## Load Data into Pandas DF

In [20]:
data_train = pd.read_csv("/content/drive/My Drive/data_train.csv")  
data_dev = pd.read_csv("/content/drive/My Drive/data_dev.csv")

,Unnamed: 0,source,replies,label
0,0,COVID-19 Fact:\nAre hand dryers effective in k...,"@WeatherBug They are, in fact, germ-breeding f...",0
1,1,@atruchecks when can we expect the result of m...,@ewart_lynne @atruchecks Hi have you had any l...,0
2,2,How does COVID-19 spread? \n\nPeople can catch...,I've read a lot about Corona virus lately and ...,0
3,3,"every news outlet using headlines like,\n\n""ar...","@TuckyAalto Apparently, when a headline is a q...",0
4,4,Researcher @naskrecki on his encounter with a ...,@Harvard @naskrecki eu tenho uma dessas em cas...,0
...,...,...,...,...
627,627,"or cure for COVID-19. However, there are sever...",WHAT ARE THE TREATMENT OPTIONS FOR COVID-19 (I...,0
628,628,"After speculation that he’s been arrested, Ban...",@artnet @xklamation there was a story saying s...,1
629,629,*Your questions answered*❓\n\n*Reply with the ...,s?\n\n14. Can I catch COVID-19 from infected s...,0
630,630,"►#Anonymous Operation #KKK ►Ku Klux Klan, We n...",@AnonymousVideo https://t.co/84M8wTOAoM @grann...,1


In [8]:
dataset_train = Dataset.from_pandas(data_train)
dataset_train

Dataset({
    features: ['source', 'replies', 'label'],
    num_rows: 1895
})

In [9]:
dataset_dev = Dataset.from_pandas(data_dev)
dataset_dev

Dataset({
    features: ['source', 'replies', 'label'],
    num_rows: 632
})

In [10]:
total_train = pd.concat([data_dev,data_train], axis=0)
dataset_total_train = Dataset.from_pandas(total_train)

## Transformer Dataset to TensorFlow TF Dataset

Makin use of Transformer Dataset

In [11]:
## Set up Tokenizer


tokenizer = AutoTokenizer.from_pretrained(BERT_MODEL, normalization=True)

## Tokenize with two sentences separated by [SEP]m, use source and reply as two sentences
def tokenize_function(dataset):
    return tokenizer(dataset["source"], dataset["replies"], padding="max_length", truncation=True)

# Set up Train data
tokenized_train_datasets = dataset_train.map(tokenize_function, batched=True)
# Set up Dev Data
tokenized_dev_datasets = dataset_dev.map(tokenize_function, batched=True)


# Set up Total Data
tokenized_total_datasets = dataset_total_train.map(tokenize_function, batched=True)


100%|██████████| 3/3 [00:00<00:00,  6.96ba/s]


In [12]:
## Use data_collator to batch the dataset
data_collator = DefaultDataCollator(return_tensors="tf")

In [13]:
tf_train_dataset = tokenized_train_datasets.to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["label"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=BATCH_SIZE,
)

tf_validation_dataset = tokenized_dev_datasets.to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["label"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=BATCH_SIZE,
)



tf_totaltrain_dataset = tokenized_total_datasets.to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["label"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=BATCH_SIZE,
)

2022-05-09 13:18:07.556081: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-09 13:18:07.577223: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-09 13:18:07.577325: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-09 13:18:07.577983: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

## Tensor Flow Bert Model

Use Training Set to train and test against dev set

In [14]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [15]:
# https://stackoverflow.com/questions/52041931/is-there-an-optimizer-in-keras-based-on-precision-or-recall-instead-of-loss
from keras import backend as K
THRESHOLD = 0.5
def precision(y_true, y_pred, threshold_shift=0.5-THRESHOLD):

    # just in case 
    y_pred = K.clip(y_pred, 0, 1)

    # shifting the prediction threshold from .5 if needed
    y_pred_bin = K.round(y_pred + threshold_shift)

    tp = K.sum(K.round(y_true * y_pred_bin)) + K.epsilon()
    fp = K.sum(K.round(K.clip(y_pred_bin - y_true, 0, 1)))

    precision = tp / (tp + fp)
    return precision


def recall(y_true, y_pred, threshold_shift=0.5-THRESHOLD):

    # just in case 
    y_pred = K.clip(y_pred, 0, 1)

    # shifting the prediction threshold from .5 if needed
    y_pred_bin = K.round(y_pred + threshold_shift)

    tp = K.sum(K.round(y_true * y_pred_bin)) + K.epsilon()
    fn = K.sum(K.round(K.clip(y_true - y_pred_bin, 0, 1)))

    recall = tp / (tp + fn)
    return recall


def fbeta(y_true, y_pred, beta = 2, threshold_shift=0.5-THRESHOLD):   
    # just in case 
    y_pred = K.clip(y_pred, 0, 1)

    # shifting the prediction threshold from .5 if needed
    y_pred_bin = K.round(y_pred + threshold_shift)

    tp = K.sum(K.round(y_true * y_pred_bin)) + K.epsilon()
    fp = K.sum(K.round(K.clip(y_pred_bin - y_true, 0, 1)))
    fn = K.sum(K.round(K.clip(y_true - y_pred, 0, 1)))

    precision = tp / (tp + fp)
    recall = tp / (tp + fn)

    beta_squared = beta ** 2
    return (beta_squared + 1) * (precision * recall) / (beta_squared * precision + recall) 


In [16]:


## Define Model
# model = TFAutoModelForSequenceClassification.from_pretrained(BERT_MODEL, num_labels=2,
#         attention_probs_dropout_prob=DROPOUT,
#         hidden_dropout_prob=DROPOUT)

model = TFAutoModelForSequenceClassification.from_pretrained(BERT_MODEL, num_labels=2)


## Set up optimisation method, minimise which loss
# model.compile(
#     optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
#     loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
#     metrics=tf.metrics.SparseCategoricalAccuracy(),
# )

# Use f1 score to capture information
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
    metrics=[tf.metrics.SparseCategoricalAccuracy(), fbeta,precision,recall]
)


2022-05-09 13:18:11.430368: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
# Fit model
model.fit(tf_train_dataset, validation_data=tf_validation_dataset, epochs=NUM_EPOCHS)


Epoch 1/6


2022-05-09 13:18:28.801026: W tensorflow/core/common_runtime/bfc_allocator.cc:462] Allocator (GPU_0_bfc) ran out of memory trying to allocate 120.00MiB (rounded to 125829120)requested by op tf_bert_for_sequence_classification/bert/encoder/layer_._4/intermediate/Gelu/Erf
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2022-05-09 13:18:28.801116: I tensorflow/core/common_runtime/bfc_allocator.cc:1010] BFCAllocator dump for GPU_0_bfc
2022-05-09 13:18:28.801140: I tensorflow/core/common_runtime/bfc_allocator.cc:1017] Bin (256): 	Total Chunks: 112, Chunks in use: 110. 28.0KiB allocated for chunks. 27.5KiB in use in bin. 810B client-requested in use in bin.
2022-05-09 13:18:28.801154: I tensorflow/core/common_runtime/bfc_allocator.cc:1017] Bin (512): 	Total Chunks: 0, Chunks in use: 0. 0B allocated for chunks. 0B in use in bin. 0B clien

ResourceExhaustedError: Graph execution error:

Detected at node 'tf_bert_for_sequence_classification/bert/encoder/layer_._4/intermediate/Gelu/Erf' defined at (most recent call last):
    File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "/home/lee/.local/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/home/lee/.local/lib/python3.10/site-packages/traitlets/config/application.py", line 846, in launch_instance
      app.start()
    File "/home/lee/.local/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 712, in start
      self.io_loop.start()
    File "/home/lee/.local/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "/usr/lib/python3.10/asyncio/base_events.py", line 

## Train using Train + Dev Set

In [ ]:
# Fit model
#model.fit(tf_totaltrain_dataset, validation_data=tf_validation_dataset, epochs=NUM_EPOCHS)


Epoch 1/4
315/315 [==============================] - 102s 284ms/step - loss: 0.3339 - sparse_categorical_accuracy: 0.8492 - val_loss: 0.1402 - val_sparse_categorical_accuracy: 0.9494
Epoch 2/4
315/315 [==============================] - 93s 294ms/step - loss: 0.1444 - sparse_categorical_accuracy: 0.9504 - val_loss: 0.0735 - val_sparse_categorical_accuracy: 0.9778
Epoch 3/4
315/315 [==============================] - 93s 296ms/step - loss: 0.0943 - sparse_categorical_accuracy: 0.9679 - val_loss: 0.0865 - val_sparse_categorical_accuracy: 0.9684
Epoch 4/4
315/315 [==============================] - 93s 296ms/step - loss: 0.0649 - sparse_categorical_accuracy: 0.9806 - val_loss: 0.0211 - val_sparse_categorical_accuracy: 0.9953


## Test on Test Set

In [ ]:
test_df = bert_preprocess(X_TEST_ID_PATH, TEST_DATA_PATH)
test_dataset = Dataset.from_pandas(test_df)  # Convert to Transformer Dataset

In [ ]:
# Convert to Keras input for Bert model
tokenized_test_datasets = test_dataset.map(tokenize_function, batched=True)
tf_test_dataset = tokenized_test_datasets.to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=BATCH_SIZE,
)

100%|██████████| 1/1 [00:00<00:00, 10.22ba/s]


In [ ]:
y_pred = model.predict(tf_test_dataset)

In [ ]:
# generate the csv for prediction
def generate_csv(pred, csv_name):
    ids = pd.Index(range(len(pred)), name='Id')
    predictions = pd.DataFrame(pred, index=ids)
    predictions.columns = ['Predicted']
    predictions.to_csv(csv_name)

def model_output_to_label(model_output):
    """Conver the output class of a tensorflow model to label"""
    logit_df = pd.DataFrame(model_output.to_tuple()[0], columns = ["0","1"])
    ## Choose highest logit as the predicted class
    logit_df["label"] = logit_df.apply(lambda x: 0 if x["0"] > x["1"] else 1, axis=1)
    return logit_df["label"]

In [ ]:
labels = model_output_to_label(y_pred)
generate_csv(labels, "/content/drive/My Drive/vanillabertmodel.csv")